# BNP Paribas Securities Services - Category Analysis

**Problématique** : *How can we improve our institutional clients' day-to-day experience?*

**Période d'analyse** : Janvier à Septembre 2024-2025

**Approche** :
1. Analyse de la répartition par catégorie de demandes
2. Temps de traitement par catégorie (impact sur l'expérience client)
3. Tendances temporelles et patterns
4. Identification des pain points clients
5. KPIs

---
## 1. Configuration & Connexion

In [71]:
import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuration
BASE_DIR = Path('/Users/jo/Desktop/BNP Paribas')
DB_PATH = BASE_DIR / 'Data/Processed/hobart_database.db'
EXPORT_DIR = BASE_DIR / 'Data/Exports'
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

# Connexion
conn = sqlite3.connect(str(DB_PATH))

print("✅ Connexion établie à la base de données Hobart")
print(f"📂 Base: {DB_PATH}")
print(f"📊 Période: Janvier - Septembre 2024-2025")

# Configuration Plotly
import plotly.io as pio
pio.templates.default = "plotly_white"

✅ Connexion établie à la base de données Hobart
📂 Base: /Users/jo/Desktop/BNP Paribas/Data/Processed/hobart_database.db
📊 Période: Janvier - Septembre 2024-2025


---
## 2. Vue d'Ensemble : Statistiques Clés

Comprendre le volume et la nature des demandes clients.

In [72]:
# Statistiques globales (Jan-Sep uniquement, exclure Octobre incomplet)
stats_global = pd.read_sql_query("""
SELECT 
    COUNT(*) as total_srs,
    COUNT(CASE WHEN CLOSINGDATE IS NOT NULL THEN 1 END) as srs_closed,
    ROUND(COUNT(CASE WHEN CLOSINGDATE IS NOT NULL THEN 1 END) * 100.0 / COUNT(*), 2) as closure_rate,
    COUNT(DISTINCT CATEGORY_ID) as categories_count,
    COUNT(DISTINCT JUR_DESK_ID) as desks_count,
    ROUND(AVG(CAST((julianday(CLOSINGDATE) - julianday(CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close
FROM sr

""", conn)

print("="*70)
print("📊 VUE D'ENSEMBLE (Janvier - Septembre)")
print("="*70)
print(f"\n📋 Service Requests totaux : {stats_global['total_srs'][0]:,}")
print(f"✅ SRs fermés : {stats_global['srs_closed'][0]:,} ({stats_global['closure_rate'][0]}%)")
print(f"📂 Catégories de demandes : {stats_global['categories_count'][0]}")
print(f"👥 Desks actifs : {stats_global['desks_count'][0]}")
print(f"⏱️  Temps moyen de résolution : {stats_global['avg_hours_to_close'][0]:.1f} heures ({stats_global['avg_hours_to_close'][0]/24:.1f} jours)")

display(stats_global)

📊 VUE D'ENSEMBLE (Janvier - Septembre)

📋 Service Requests totaux : 2,370,662
✅ SRs fermés : 2,314,348 (97.62%)
📂 Catégories de demandes : 1565
👥 Desks actifs : 877
⏱️  Temps moyen de résolution : 167.2 heures (7.0 jours)


,total_srs,srs_closed,closure_rate,categories_count,desks_count,avg_hours_to_close
0,2370662,2314348,97.62,1565,877,167.17


---
## 3. Analyse par Catégorie : Répartition des Demandes Clients

**Question Clé** : Quelles sont les catégories de demandes les plus fréquentes ?

In [73]:
# Répartition par catégorie (Top 20)
category_distribution = pd.read_sql_query("""
SELECT 
    c.NAME as category_name,
    c.ID as category_id,
    COUNT(sr.ID) as total_requests,
    COUNT(CASE WHEN sr.CLOSINGDATE IS NOT NULL THEN 1 END) as closed_requests,
    ROUND(COUNT(CASE WHEN sr.CLOSINGDATE IS NOT NULL THEN 1 END) * 100.0 / COUNT(sr.ID), 2) as closure_rate,
    ROUND(AVG(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close,
    ROUND(COUNT(sr.ID) * 100.0 / (SELECT COUNT(*) FROM sr ), 2) as pct_of_total
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
WHERE strftime('%Y-%m', sr.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY c.ID, c.NAME
ORDER BY total_requests DESC
LIMIT 20
""", conn)

print("="*70)
print("📂 TOP 20 CATÉGORIES DE DEMANDES CLIENTS")
print("="*70)
print(f"\nTop 5 catégories représentent {category_distribution.head(5)['pct_of_total'].sum():.1f}% du volume total\n")

display(category_distribution)

📂 TOP 20 CATÉGORIES DE DEMANDES CLIENTS

Top 5 catégories représentent 9.0% du volume total



,category_name,category_id,total_requests,closed_requests,closure_rate,avg_hours_to_close,pct_of_total
0,Cash instruction,353585,65780,65677,99.84,62.71,2.77
1,Tax,172255,48176,45329,94.09,583.16,2.03
2,BAU Asset Creation,218638,34432,34407,99.93,23.64,1.45
3,OTHERS,154192,33615,33447,99.50,84.72,1.42
4,CREST,371413,31853,31816,99.88,39.36,1.34
5,Settlement,173197,29441,29277,99.44,139.13,1.24
6,Investigation Level 1,219273,29391,29357,99.88,84.65,1.24
7,FRPP,327439,27565,27509,99.80,83.38,1.16
8,DEFF,327494,25333,25306,99.89,49.59,1.07
9,Transaction cycle and follow-up,478777,25179,25174,99.98,61.44,1.06


In [74]:
# Visualisation : Répartition par catégorie
fig = px.bar(
    category_distribution.head(15),
    x='category_name',
    y='total_requests',
    title='Top 15 Catégories de Demandes Clients (Jan-Sep)',
    labels={'category_name': 'Catégorie', 'total_requests': 'Nombre de Demandes'},
    text='total_requests',
    color='closure_rate',
    color_continuous_scale='RdYlGn',
    hover_data=['avg_hours_to_close', 'pct_of_total']
)

fig.update_traces(textposition='outside')
fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

---
## 4. Temps de Traitement par Catégorie : Impact sur l'Expérience Client

**Question Clé** : Quelles catégories ont les temps de traitement les plus longs (pain points clients) ?

In [75]:
# Temps de traitement par catégorie (catégories avec > 100 SRs)
treatment_time_by_category = pd.read_sql_query("""
SELECT 
    c.NAME as category_name,
    COUNT(sr.ID) as total_requests,
    ROUND(AVG(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours,
    ROUND(MIN(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as min_hours,
    ROUND(MAX(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as max_hours,
    ROUND(AVG(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)) / 24, 1) as avg_days
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
WHERE sr.CLOSINGDATE IS NOT NULL 
  AND sr.CREATIONDATE IS NOT NULL

GROUP BY c.NAME
HAVING COUNT(sr.ID) > 100
ORDER BY avg_hours DESC
LIMIT 20
""", conn)

print("="*70)
print("⏱️  TEMPS DE TRAITEMENT PAR CATÉGORIE (Top 20 plus longs)")
print("="*70)
print("\n⚠️  Catégories avec les temps les plus longs = Pain points clients !\n")

display(treatment_time_by_category)

⏱️  TEMPS DE TRAITEMENT PAR CATÉGORIE (Top 20 plus longs)

⚠️  Catégories avec les temps les plus longs = Pain points clients !



,category_name,total_requests,avg_hours,min_hours,max_hours,avg_days
0,SUPPORT,113,2888.68,0.03,16534.34,120.4
1,CLIENT PROJECT,126,2673.19,0.01,10219.10,111.4
2,Breakdown Tax Reclaims,138,2567.11,0.01,22222.44,107.0
3,EXTERNAL QUERIES,5693,2217.93,0.00,13965.28,92.4
4,Matrix tool,107,2058.45,2.14,12402.02,85.8
5,INTERNAL QUERIES,5503,2051.19,0.00,15426.08,85.5
6,Tax Voucher,106,1892.96,0.02,11419.93,78.9
7,Tax Reclaim,832,1888.50,0.01,15213.55,78.7
8,General client queries,129,1818.36,0.01,12984.61,75.8
9,TAX INCOME-TAX,1535,1781.43,0.00,18985.95,74.2


In [76]:
# Visualisation : Temps moyen par catégorie
fig = go.Figure()

# Trier par temps de traitement
df_sorted = treatment_time_by_category.head(15).sort_values('avg_days')

fig.add_trace(go.Bar(
    y=df_sorted['category_name'],
    x=df_sorted['avg_days'],
    orientation='h',
    text=df_sorted['avg_days'].apply(lambda x: f"{x:.1f}j"),
    textposition='outside',
    marker_color=df_sorted['avg_days'],
    marker_colorscale='Reds',
    hovertemplate='<b>%{y}</b><br>Temps moyen: %{x:.1f} jours<extra></extra>'
))

fig.update_layout(
    title='Temps Moyen de Traitement par Catégorie (en jours)',
    xaxis_title='Jours',
    yaxis_title='Catégorie',
    height=600,
    showlegend=False
)

fig.show()

---
## 5. Tendances Temporelles : Évolution des Demandes par Catégorie

**Question Clé** : Comment évoluent les demandes clients dans le temps ?

In [77]:
# Évolution mensuelle des top 10 catégories
top_10_categories = category_distribution.head(10)['category_id'].tolist()

monthly_category_trends = pd.read_sql_query(f"""
SELECT 
    strftime('%Y-%m', sr.CREATIONDATE) as month,
    c.NAME as category_name,
    COUNT(sr.ID) as total_requests,
    ROUND(AVG(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
WHERE sr.CATEGORY_ID IN ({','.join(map(str, top_10_categories))})

GROUP BY month, c.NAME
ORDER BY month, total_requests DESC
""", conn)

print("="*70)
print("📈 ÉVOLUTION MENSUELLE DES TOP 10 CATÉGORIES")
print("="*70)
print(f"Nombre de mois analysés : {monthly_category_trends['month'].nunique()}\n")

display(monthly_category_trends.head(30))

📈 ÉVOLUTION MENSUELLE DES TOP 10 CATÉGORIES
Nombre de mois analysés : 36



,month,category_name,total_requests,avg_hours_to_close
0,2023-01,Tax,4,25415.61
1,2023-01,FRPP,1,NaN
2,2023-02,Tax,3,25193.82
3,2023-02,Settlement,1,NaN
4,2023-03,Tax,5,24487.65
5,2023-04,Tax,3,NaN
6,2023-04,CREST,1,23565.20
7,2023-05,Tax,6,22838.08
8,2023-06,Tax,3,22153.02
9,2023-07,Tax,6,19282.15


In [78]:
# Visualisation : Évolution des top 5 catégories
top_5_categories_names = category_distribution.head(5)['category_name'].tolist()
df_top5 = monthly_category_trends[monthly_category_trends['category_name'].isin(top_5_categories_names)]

fig = px.line(
    df_top5,
    x='month',
    y='total_requests',
    color='category_name',
    title='Évolution Mensuelle des Top 5 Catégories de Demandes',
    labels={'month': 'Mois', 'total_requests': 'Nombre de Demandes', 'category_name': 'Catégorie'},
    markers=True
)

fig.update_layout(height=500, hovermode='x unified')
fig.show()

---
## 6. KPIs Orientés Expérience Client

**Focus** : SLA, First Response Time, Taux de Résolution

In [79]:
# KPIs expérience client par catégorie
client_experience_kpis = pd.read_sql_query("""
SELECT 
    c.NAME as category_name,
    COUNT(sr.ID) as total_requests,
    
    -- Taux de résolution
    ROUND(COUNT(CASE WHEN sr.CLOSINGDATE IS NOT NULL THEN 1 END) * 100.0 / COUNT(sr.ID), 2) as resolution_rate,
    
    -- Temps de première réponse (Acknowledge)
    ROUND(AVG(CAST((julianday(sr.ACKNOWLEDGE_DATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as avg_first_response_hours,
    
    -- Temps de traitement total
    ROUND(AVG(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as avg_total_hours,
    
    -- Respect SLA (expiration_date)
    ROUND(COUNT(CASE WHEN sr.CLOSINGDATE <= sr.EXPIRATION_DATE THEN 1 END) * 100.0 / 
          COUNT(CASE WHEN sr.CLOSINGDATE IS NOT NULL AND sr.EXPIRATION_DATE IS NOT NULL THEN 1 END), 2) as sla_compliance,
    
    -- Nombre de contacts moyen
    ROUND(AVG((SELECT COUNT(*) FROM srcontact WHERE srcontact.SR_ID = sr.ID)), 2) as avg_contacts
    
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
WHERE strftime('%Y-%m', sr.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY c.NAME
HAVING COUNT(sr.ID) > 100
ORDER BY total_requests DESC
LIMIT 20
""", conn)

print("="*70)
print("🎯 KPIs EXPÉRIENCE CLIENT PAR CATÉGORIE")
print("="*70)
print("\nIndicateurs clés :")
print("  • Taux de résolution : % de SRs fermés")
print("  • Premier temps de réponse : Temps avant acknowledge")
print("  • SLA Compliance : % de SRs traités avant expiration")
print("  • Contacts moyens : Nombre d'interactions par SR\n")

display(client_experience_kpis)

🎯 KPIs EXPÉRIENCE CLIENT PAR CATÉGORIE

Indicateurs clés :
  • Taux de résolution : % de SRs fermés
  • Premier temps de réponse : Temps avant acknowledge
  • SLA Compliance : % de SRs traités avant expiration
  • Contacts moyens : Nombre d'interactions par SR



,category_name,total_requests,resolution_rate,avg_first_response_hours,avg_total_hours,sla_compliance,avg_contacts
0,Tax,67431,95.02,9.63,584.63,63.22,7.62
1,Cash instruction,65780,99.84,1.66,62.71,81.07,4.02
2,BAU Asset Creation,34432,99.93,NaN,23.64,39.36,2.74
3,OTHERS,33615,99.50,3.20,84.72,72.65,3.58
4,Settlement,32320,99.45,0.79,142.22,67.27,7.46
5,CREST,31853,99.88,0.45,39.36,85.12,3.37
6,Investigation Level 1,29391,99.88,1.95,84.65,73.16,5.24
7,FRPP,27565,99.80,0.46,83.38,66.74,6.07
8,DEFF,25333,99.89,0.39,49.59,74.93,4.77
9,Transaction cycle and follow-up,25179,99.98,0.29,61.44,93.66,5.70


In [80]:
# Visualisation : SLA Compliance vs Volume (Top 15)
df_viz = client_experience_kpis.head(15).copy()

fig = px.scatter(
    df_viz,
    x='total_requests',
    y='sla_compliance',
    size='avg_total_hours',
    color='resolution_rate',
    hover_name='category_name',
    title='SLA Compliance vs Volume (Top 15 Catégories)',
    labels={
        'total_requests': 'Volume de Demandes',
        'sla_compliance': 'SLA Compliance (%)',
        'avg_total_hours': 'Temps Moyen (h)',
        'resolution_rate': 'Taux Résolution (%)'
    },
    color_continuous_scale='RdYlGn',
    size_max=60
)

fig.add_hline(y=95, line_dash="dash", line_color="red", 
              annotation_text="Target SLA 95%", annotation_position="right")

fig.update_layout(height=600)
fig.show()

---
## 7. Analyse des Pain Points Clients

**Identification** : Catégories avec faible performance sur l'expérience client

In [81]:
# Identifier les pain points (volume élevé + SLA faible OU temps long)
pain_points = client_experience_kpis[
    ((client_experience_kpis['total_requests'] > 1000) & 
     ((client_experience_kpis['sla_compliance'] < 90) | 
      (client_experience_kpis['avg_total_hours'] > 100)))
].sort_values('total_requests', ascending=False)

print("="*70)
print("⚠️  PAIN POINTS CLIENTS IDENTIFIÉS")
print("="*70)
print("\nCritères : Volume > 1000 ET (SLA < 90% OU Temps > 100h)")
print(f"\nNombre de catégories problématiques : {len(pain_points)}\n")

if len(pain_points) > 0:
    print("🔴 Catégories nécessitant une attention immédiate :\n")
    display(pain_points)
    
    # Calculer l'impact total
    total_impact = pain_points['total_requests'].sum()
    total_srs = category_distribution['total_requests'].sum()
    impact_pct = (total_impact / total_srs) * 100
    
    print(f"\n💥 Impact : {total_impact:,} SRs concernés ({impact_pct:.1f}% du volume total)")
else:
    print("✅ Aucun pain point majeur identifié avec ces critères")

⚠️  PAIN POINTS CLIENTS IDENTIFIÉS

Critères : Volume > 1000 ET (SLA < 90% OU Temps > 100h)

Nombre de catégories problématiques : 19

🔴 Catégories nécessitant une attention immédiate :



,category_name,total_requests,resolution_rate,avg_first_response_hours,avg_total_hours,sla_compliance,avg_contacts
0,Tax,67431,95.02,9.63,584.63,63.22,7.62
1,Cash instruction,65780,99.84,1.66,62.71,81.07,4.02
2,BAU Asset Creation,34432,99.93,NaN,23.64,39.36,2.74
3,OTHERS,33615,99.50,3.20,84.72,72.65,3.58
4,Settlement,32320,99.45,0.79,142.22,67.27,7.46
5,CREST,31853,99.88,0.45,39.36,85.12,3.37
6,Investigation Level 1,29391,99.88,1.95,84.65,73.16,5.24
7,FRPP,27565,99.80,0.46,83.38,66.74,6.07
8,DEFF,25333,99.89,0.39,49.59,74.93,4.77
10,KPI PARIS,22410,99.98,NaN,6.05,52.03,2.31



💥 Impact : 558,062 SRs concernés (99.5% du volume total)


---
## 8. Analyse des Réouvertures (Indicateur de Satisfaction)

**Question** : Quelles catégories ont le plus de SRs réouverts (insatisfaction client) ?

In [82]:
# Analyse des réouvertures par catégorie
reopened_analysis = pd.read_sql_query("""
SELECT 
    c.NAME as category_name,
    COUNT(sr.ID) as total_requests,
    COUNT(CASE WHEN sr.REOPEN_DATE IS NOT NULL THEN 1 END) as reopened_count,
    ROUND(COUNT(CASE WHEN sr.REOPEN_DATE IS NOT NULL THEN 1 END) * 100.0 / COUNT(sr.ID), 2) as reopen_rate
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
WHERE strftime('%Y-%m', sr.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY c.NAME
HAVING COUNT(sr.ID) > 100 AND reopened_count > 0
ORDER BY reopen_rate DESC
LIMIT 20
""", conn)

print("="*70)
print("🔄 ANALYSE DES RÉOUVERTURES (Indicateur de Satisfaction)")
print("="*70)
print("\n⚠️  Taux de réouverture élevé = Résolution incomplète ou insatisfaction\n")

if len(reopened_analysis) > 0:
    display(reopened_analysis)
    
    avg_reopen_rate = reopened_analysis['reopen_rate'].mean()
    print(f"\n📊 Taux moyen de réouverture : {avg_reopen_rate:.2f}%")
else:
    print("✅ Très peu de réouvertures dans les données")

🔄 ANALYSE DES RÉOUVERTURES (Indicateur de Satisfaction)

⚠️  Taux de réouverture élevé = Résolution incomplète ou insatisfaction



,category_name,total_requests,reopened_count,reopen_rate
0,Cannis,195,194,99.49
1,Breakdown RAS,312,310,99.36
2,Processing,247,219,88.66
3,Trade Placement,1469,1263,85.98
4,Physicals,164,136,82.93
5,OST,753,603,80.08
6,Support to Production,352,278,78.98
7,Aberdeen Case,107,84,78.50
8,Requests to HUB,269,210,78.07
9,Settlement Trade Format,209,153,73.21



📊 Taux moyen de réouverture : 75.65%


---
## 9. nombre de SR par catégorie et combien d’entre elles ont eu besoin de faire une activité

In [83]:
# Analyse des SRs avec activités par catégorie
activity_analysis = pd.read_sql_query("""
SELECT 
    c.NAME as category_name,
    COUNT(DISTINCT sr.ID) as total_srs,
    COUNT(DISTINCT CASE WHEN activity.ID IS NOT NULL THEN sr.ID END) as srs_with_activity,
    ROUND(COUNT(DISTINCT CASE WHEN activity.ID IS NOT NULL THEN sr.ID END) * 100.0 / COUNT(DISTINCT sr.ID), 2) as activity_rate,
    COUNT(activity.ID) as total_activities,
    ROUND(AVG(CASE WHEN activity.ID IS NOT NULL THEN 1.0 * (SELECT COUNT(*) FROM activity a2 WHERE a2.SR_ID = sr.ID) END), 2) as avg_activities_per_sr,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)), 2) as avg_activity_hours,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)) / 24, 1) as avg_activity_days
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
LEFT JOIN activity ON sr.ID = activity.SR_ID
WHERE strftime('%Y-%m', sr.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY c.ID, c.NAME
HAVING COUNT(DISTINCT sr.ID) > 100
ORDER BY total_srs DESC
LIMIT 20
""", conn)

print("="*70)
print("📋 ANALYSE DES ACTIVITÉS PAR CATÉGORIE")
print("="*70)
print("\nIndicateurs :")
print("  • Total SRs : Nombre total de demandes")
print("  • SRs avec activité : Nombre de SRs nécessitant une intervention manuelle")
print("  • Taux d'activité : % de SRs nécessitant une activité")
print("  • Activités moyennes : Nombre moyen d'activités par SR")
print("  • Temps moyen activité : Durée moyenne de traitement d'une activité\n")

display(activity_analysis)

# Statistiques globales
total_with_activity = activity_analysis['srs_with_activity'].sum()
total_all = activity_analysis['total_srs'].sum()
global_activity_rate = (total_with_activity / total_all) * 100
avg_activity_time = activity_analysis['avg_activity_hours'].mean()

print(f"\n📊 Statistiques globales (Top 20 catégories) :")
print(f"   • SRs avec activité : {total_with_activity:,} / {total_all:,}")
print(f"   • Taux global d'activité : {global_activity_rate:.2f}%")
print(f"   • Temps moyen d'une activité : {avg_activity_time:.1f}h ({avg_activity_time/24:.1f}j)")

📋 ANALYSE DES ACTIVITÉS PAR CATÉGORIE

Indicateurs :
  • Total SRs : Nombre total de demandes
  • SRs avec activité : Nombre de SRs nécessitant une intervention manuelle
  • Taux d'activité : % de SRs nécessitant une activité
  • Activités moyennes : Nombre moyen d'activités par SR
  • Temps moyen activité : Durée moyenne de traitement d'une activité



,category_name,total_srs,srs_with_activity,activity_rate,total_activities,avg_activities_per_sr,avg_activity_hours,avg_activity_days
0,Cash instruction,65780,5319,8.09,7227,2.45,1.23,0.1
1,Tax,48176,4020,8.34,8214,8.10,95.46,4.0
2,BAU Asset Creation,34432,6,0.02,10,2.40,4.09,0.2
3,OTHERS,33615,916,2.72,1122,1.96,156.04,6.5
4,CREST,31853,14481,45.46,17771,1.82,1.24,0.1
5,Settlement,29441,32,0.11,39,1.67,186.35,7.8
6,Investigation Level 1,29391,1680,5.72,1997,1.56,1.36,0.1
7,FRPP,27565,2970,10.77,4491,3.06,0.69,0.0
8,DEFF,25333,3419,13.50,4848,2.04,0.42,0.0
9,Transaction cycle and follow-up,25179,164,0.65,213,1.80,1.76,0.1



📊 Statistiques globales (Top 20 catégories) :
   • SRs avec activité : 43,705 / 561,107
   • Taux global d'activité : 7.79%
   • Temps moyen d'une activité : 39.8h (1.7j)


In [84]:
# Visualisation : Taux d'activité par catégorie
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Taux d\'Activité par Catégorie (Top 15)', 'Volume SRs vs Taux d\'Activité'),
    specs=[[{"type": "bar"}, {"type": "scatter"}]]
)

df_viz = activity_analysis.head(15).copy()
# Remplacer les NaN par 1 pour éviter les erreurs dans la taille des points
df_viz['avg_activities_per_sr'] = df_viz['avg_activities_per_sr'].fillna(1)

# Graphique 1 : Barres horizontales du taux d'activité
fig.add_trace(
    go.Bar(
        y=df_viz['category_name'],
        x=df_viz['activity_rate'],
        orientation='h',
        text=df_viz['activity_rate'].apply(lambda x: f"{x:.1f}%"),
        textposition='outside',
        marker_color=df_viz['activity_rate'],
        marker_colorscale='Blues',
        name='Taux activité',
        hovertemplate='<b>%{y}</b><br>Taux: %{x:.1f}%<extra></extra>'
    ),
    row=1, col=1
)

# Graphique 2 : Scatter volume vs taux d'activité
fig.add_trace(
    go.Scatter(
        x=df_viz['total_srs'],
        y=df_viz['activity_rate'],
        mode='markers+text',
        marker=dict(
            size=df_viz['avg_activities_per_sr']*10,
            color=df_viz['activity_rate'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Taux<br>Activité", x=1.15)
        ),
        text=df_viz['category_name'].str[:15],
        textposition='top center',
        hovertemplate='<b>%{text}</b><br>Volume: %{x:,}<br>Taux activité: %{y:.1f}%<extra></extra>',
        name='Catégories'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Taux d'Activité (%)", row=1, col=1)
fig.update_yaxes(title_text="Catégorie", row=1, col=1)
fig.update_xaxes(title_text="Nombre de SRs", row=1, col=2)
fig.update_yaxes(title_text="Taux d'Activité (%)", row=1, col=2)

fig.update_layout(height=500, showlegend=False, title_text="Analyse des Activités par Catégorie")
fig.show()

---
## 10. Analyse Détaillée des Activités

**Focus** : Statistiques sur les activités elles-mêmes (types, durées, volumes)

In [85]:
# Statistiques globales sur les activités
activity_stats = pd.read_sql_query("""
SELECT 
    COUNT(*) as total_activities,
    COUNT(DISTINCT SR_ID) as unique_srs_with_activities,
    COUNT(CASE WHEN CLOSINGDATE IS NOT NULL THEN 1 END) as closed_activities,
    ROUND(COUNT(CASE WHEN CLOSINGDATE IS NOT NULL THEN 1 END) * 100.0 / COUNT(*), 2) as closure_rate,
    ROUND(AVG(CAST((julianday(CLOSINGDATE) - julianday(CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close,
    ROUND(MIN(CAST((julianday(CLOSINGDATE) - julianday(CREATIONDATE)) * 24 AS REAL)), 2) as min_hours,
    ROUND(MAX(CAST((julianday(CLOSINGDATE) - julianday(CREATIONDATE)) * 24 AS REAL)), 2) as max_hours
FROM activity

""", conn)

print("="*70)
print("📊 STATISTIQUES GLOBALES DES ACTIVITÉS")
print("="*70)
print(f"\n📋 Total d'activités : {activity_stats['total_activities'][0]:,}")
print(f"🎯 SRs concernés : {activity_stats['unique_srs_with_activities'][0]:,}")
print(f"✅ Activités fermées : {activity_stats['closed_activities'][0]:,} ({activity_stats['closure_rate'][0]}%)")
print(f"⏱️  Temps moyen de traitement : {activity_stats['avg_hours_to_close'][0]:.1f}h ({activity_stats['avg_hours_to_close'][0]/24:.1f}j)")
print(f"⚡ Temps min : {activity_stats['min_hours'][0]:.2f}h | Temps max : {activity_stats['max_hours'][0]:.1f}h ({activity_stats['max_hours'][0]/24:.0f}j)")

display(activity_stats)

📊 STATISTIQUES GLOBALES DES ACTIVITÉS

📋 Total d'activités : 183,248
🎯 SRs concernés : 127,277
✅ Activités fermées : 179,750 (98.09%)
⏱️  Temps moyen de traitement : 46.3h (1.9j)
⚡ Temps min : 0.00h | Temps max : 15241.8h (635j)


,total_activities,unique_srs_with_activities,closed_activities,closure_rate,avg_hours_to_close,min_hours,max_hours
0,183248,127277,179750,98.09,46.28,0.0,15241.79


In [86]:
# Analyse par type d'activité (par TYPE_ID)
activity_by_type = pd.read_sql_query("""
SELECT 
    COALESCE(activity.TYPE_ID, 0) as type_id,
    COUNT(activity.ID) as total_activities,
    COUNT(DISTINCT activity.SR_ID) as unique_srs,
    ROUND(COUNT(activity.ID) * 100.0 / (SELECT COUNT(*) FROM activity ), 2) as pct_of_total,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)) / 24, 1) as avg_days_to_close
FROM activity
WHERE strftime('%Y-%m', activity.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY activity.TYPE_ID
ORDER BY total_activities DESC
LIMIT 20
""", conn)

# Ajouter un nom lisible
activity_by_type['type_name'] = 'Type ' + activity_by_type['type_id'].astype(str)

print("="*70)
print("📊 ACTIVITÉS PAR TYPE (Top 20)")
print("="*70)
print(f"\nTop 5 types représentent {activity_by_type.head(5)['pct_of_total'].sum():.1f}% des activités\n")

display(activity_by_type)

📊 ACTIVITÉS PAR TYPE (Top 20)

Top 5 types représentent 86.0% des activités



,type_id,total_activities,unique_srs,pct_of_total,avg_hours_to_close,avg_days_to_close,type_name
0,1579,157337,108571,85.86,48.52,2.0,Type 1579
1,1580,232,232,0.13,157.15,6.5,Type 1580


In [87]:
# Analyse par statut d'activité (par STATUS_ID)
activity_by_status = pd.read_sql_query("""
SELECT 
    COALESCE(activity.STATUS_ID, 0) as status_id,
    COUNT(activity.ID) as total_activities,
    COUNT(DISTINCT activity.SR_ID) as unique_srs,
    ROUND(COUNT(activity.ID) * 100.0 / (SELECT COUNT(*) FROM activity ), 2) as pct_of_total,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)) / 24, 1) as avg_days_to_close
FROM activity
WHERE strftime('%Y-%m', activity.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY activity.STATUS_ID
ORDER BY total_activities DESC
LIMIT 20
""", conn)

# Ajouter un nom lisible
activity_by_status['status_name'] = 'Status ' + activity_by_status['status_id'].astype(str)

print("="*70)
print("📊 ACTIVITÉS PAR STATUT (Top 20)")
print("="*70)
print(f"\nTop 5 statuts représentent {activity_by_status.head(5)['pct_of_total'].sum():.1f}% des activités\n")

display(activity_by_status)

📊 ACTIVITÉS PAR STATUT (Top 20)

Top 5 statuts représentent 86.0% des activités



,status_id,total_activities,unique_srs,pct_of_total,avg_hours_to_close,avg_days_to_close,status_name
0,31,149200,104258,81.42,42.62,1.8,Status 31
1,11,6111,5334,3.33,188.70,7.9,Status 11
2,100,1566,1381,0.85,5.97,0.2,Status 100
3,10,554,483,0.30,825.67,34.4,Status 10
4,12,122,89,0.07,700.24,29.2,Status 12
5,101,10,10,0.01,0.11,0.0,Status 101
6,13,3,3,0.00,0.06,0.0,Status 13
7,14,3,3,0.00,0.05,0.0,Status 14


In [88]:
# Évolution mensuelle des activités
monthly_activities = pd.read_sql_query("""
SELECT 
    strftime('%Y-%m', activity.CREATIONDATE) as month,
    COUNT(activity.ID) as total_activities,
    COUNT(DISTINCT activity.SR_ID) as unique_srs,
    COUNT(CASE WHEN activity.CLOSINGDATE IS NOT NULL THEN 1 END) as closed_activities,
    ROUND(AVG(CAST((julianday(activity.CLOSINGDATE) - julianday(activity.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_to_close
FROM activity
WHERE strftime('%Y-%m', activity.CREATIONDATE) BETWEEN '2024-01' AND '2025-09'
GROUP BY month
ORDER BY month
""", conn)

print("="*70)
print("📈 ÉVOLUTION MENSUELLE DES ACTIVITÉS")
print("="*70)
print(f"\nPériode : Janvier 2024 - Septembre 2025")
print(f"Nombre de mois : {len(monthly_activities)}\n")

display(monthly_activities)

# Visualisation de la tendance
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Volume d\'Activités par Mois', 'Temps Moyen de Traitement par Mois'),
    vertical_spacing=0.15
)

# Graphique 1 : Volume mensuel
fig.add_trace(
    go.Scatter(
        x=monthly_activities['month'],
        y=monthly_activities['total_activities'],
        mode='lines+markers',
        name='Activités',
        line=dict(color='royalblue', width=3),
        marker=dict(size=8),
        hovertemplate='<b>%{x}</b><br>Activités: %{y:,}<extra></extra>'
    ),
    row=1, col=1
)

# Graphique 2 : Temps moyen mensuel
fig.add_trace(
    go.Scatter(
        x=monthly_activities['month'],
        y=monthly_activities['avg_hours_to_close'] / 24,
        mode='lines+markers',
        name='Temps moyen',
        line=dict(color='coral', width=3),
        marker=dict(size=8),
        hovertemplate='<b>%{x}</b><br>Temps: %{y:.1f} jours<extra></extra>'
    ),
    row=2, col=1
)

fig.update_xaxes(title_text="Mois", row=2, col=1)
fig.update_yaxes(title_text="Nombre d'Activités", row=1, col=1)
fig.update_yaxes(title_text="Jours", row=2, col=1)

fig.update_layout(height=700, showlegend=False, title_text="Tendances Temporelles des Activités")
fig.show()

📈 ÉVOLUTION MENSUELLE DES ACTIVITÉS

Période : Janvier 2024 - Septembre 2025
Nombre de mois : 21



,month,total_activities,unique_srs,closed_activities,avg_hours_to_close
0,2024-01,95,48,93,175.36
1,2024-02,157,85,150,609.49
2,2024-03,172,105,166,372.01
3,2024-04,310,191,294,211.17
4,2024-05,307,192,306,233.30
5,2024-06,352,240,349,328.04
6,2024-07,566,341,555,264.98
7,2024-08,648,394,639,328.78
8,2024-09,732,439,723,218.72
9,2024-10,1201,735,1184,220.82


In [89]:
# Visualisation : Distribution des activités
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Volume d\'Activités par Statut (Top 10)', 'Temps de Traitement par Type (Top 10)'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

# Graphique 1 : Volume par statut
df_status_top = activity_by_status.head(10).sort_values('total_activities')
fig.add_trace(
    go.Bar(
        y=df_status_top['status_name'],
        x=df_status_top['total_activities'],
        orientation='h',
        text=df_status_top['total_activities'].apply(lambda x: f"{x:,}"),
        textposition='outside',
        marker_color='lightblue',
        name='Volume',
        hovertemplate='<b>%{y}</b><br>Volume: %{x:,}<extra></extra>'
    ),
    row=1, col=1
)

# Graphique 2 : Temps par type
df_type_top = activity_by_type.head(10).sort_values('avg_days_to_close')
fig.add_trace(
    go.Bar(
        y=df_type_top['type_name'],
        x=df_type_top['avg_days_to_close'],
        orientation='h',
        text=df_type_top['avg_days_to_close'].apply(lambda x: f"{x:.1f}j"),
        textposition='outside',
        marker_color=df_type_top['avg_days_to_close'],
        marker_colorscale='Reds',
        name='Durée',
        hovertemplate='<b>%{y}</b><br>Durée: %{x:.1f} jours<extra></extra>'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Nombre d'Activités", row=1, col=1)
fig.update_yaxes(title_text="Statut", row=1, col=1)
fig.update_xaxes(title_text="Jours", row=1, col=2)
fig.update_yaxes(title_text="Type", row=1, col=2)

fig.update_layout(height=500, showlegend=False, title_text="Distribution des Activités")
fig.show()

---
## 11. Synthèse


In [90]:
print("="*70)
print("📋 SYNTHÈSE EXÉCUTIVE")
print("="*70)

# Top 3 catégories par volume
print("\n1️⃣ TOP 3 CATÉGORIES PAR VOLUME (Focus d'amélioration)")
for idx, row in category_distribution.head(3).iterrows():
    print(f"   • {row['category_name']}: {row['total_requests']:,} demandes ({row['pct_of_total']}%)")
    print(f"     → Temps moyen: {row['avg_hours_to_close']/24:.1f} jours | Closure: {row['closure_rate']}%")

# Catégories les plus lentes
print("\n2️⃣ CATÉGORIES AVEC TEMPS DE TRAITEMENT LE PLUS LONG (Pain Points)")
for idx, row in treatment_time_by_category.head(3).iterrows():
    print(f"   • {row['category_name']}: {row['avg_days']:.1f} jours en moyenne")
    print(f"     → Volume: {row['total_requests']:,} demandes")

# KPIs moyens
avg_resolution = client_experience_kpis['resolution_rate'].mean()
avg_sla = client_experience_kpis['sla_compliance'].mean()
avg_first_response = client_experience_kpis['avg_first_response_hours'].mean()

print("\n3️⃣ KPIs GLOBAUX EXPÉRIENCE CLIENT")
print(f"   • Taux de résolution moyen: {avg_resolution:.1f}%")
print(f"   • SLA Compliance moyen: {avg_sla:.1f}%")
print(f"   • Temps de première réponse: {avg_first_response:.1f}h ({avg_first_response/24:.1f}j)")

📋 SYNTHÈSE EXÉCUTIVE

1️⃣ TOP 3 CATÉGORIES PAR VOLUME (Focus d'amélioration)
   • Cash instruction: 65,780 demandes (2.77%)
     → Temps moyen: 2.6 jours | Closure: 99.84%
   • Tax: 48,176 demandes (2.03%)
     → Temps moyen: 24.3 jours | Closure: 94.09%
   • BAU Asset Creation: 34,432 demandes (1.45%)
     → Temps moyen: 1.0 jours | Closure: 99.93%

2️⃣ CATÉGORIES AVEC TEMPS DE TRAITEMENT LE PLUS LONG (Pain Points)
   • SUPPORT: 120.4 jours en moyenne
     → Volume: 113 demandes
   • CLIENT PROJECT: 111.4 jours en moyenne
     → Volume: 126 demandes
   • Breakdown Tax Reclaims: 107.0 jours en moyenne
     → Volume: 138 demandes

3️⃣ KPIs GLOBAUX EXPÉRIENCE CLIENT
   • Taux de résolution moyen: 99.1%
   • SLA Compliance moyen: 73.1%
   • Temps de première réponse: 3.1h (0.1j)


---
## 12. Export des Résultats

Sauvegarder les analyses pour présentation.

In [91]:
# Export des analyses clés
timestamp = datetime.now().strftime('%Y%m%d')

# 1. Répartition par catégorie
export_1 = EXPORT_DIR / f'category_distribution_{timestamp}.csv'
category_distribution.to_csv(export_1, index=False)
print(f"✅ Exporté: {export_1.name}")

# 2. Temps de traitement
export_2 = EXPORT_DIR / f'treatment_time_by_category_{timestamp}.csv'
treatment_time_by_category.to_csv(export_2, index=False)
print(f"✅ Exporté: {export_2.name}")

# 3. KPIs expérience client
export_3 = EXPORT_DIR / f'client_experience_kpis_{timestamp}.csv'
client_experience_kpis.to_csv(export_3, index=False)
print(f"✅ Exporté: {export_3.name}")

# 4. Pain points
if len(pain_points) > 0:
    export_4 = EXPORT_DIR / f'pain_points_{timestamp}.csv'
    pain_points.to_csv(export_4, index=False)
    print(f"✅ Exporté: {export_4.name}")

# 5. Tendances mensuelles
export_5 = EXPORT_DIR / f'monthly_category_trends_{timestamp}.csv'
monthly_category_trends.to_csv(export_5, index=False)
print(f"✅ Exporté: {export_5.name}")

# 6. Analyse des activités
export_6 = EXPORT_DIR / f'activity_analysis_{timestamp}.csv'
activity_analysis.to_csv(export_6, index=False)
print(f"✅ Exporté: {export_6.name}")

# 7. Activités par statut
export_7 = EXPORT_DIR / f'activity_by_status_{timestamp}.csv'
activity_by_status.to_csv(export_7, index=False)
print(f"✅ Exporté: {export_7.name}")

# 8. Activités par type
export_8 = EXPORT_DIR / f'activity_by_type_{timestamp}.csv'
activity_by_type.to_csv(export_8, index=False)
print(f"✅ Exporté: {export_8.name}")

# 9. Évolution mensuelle des activités
export_9 = EXPORT_DIR / f'monthly_activities_{timestamp}.csv'
monthly_activities.to_csv(export_9, index=False)
print(f"✅ Exporté: {export_9.name}")

print(f"\n📂 Tous les exports dans: {EXPORT_DIR}")

✅ Exporté: category_distribution_20260215.csv
✅ Exporté: treatment_time_by_category_20260215.csv
✅ Exporté: client_experience_kpis_20260215.csv
✅ Exporté: pain_points_20260215.csv
✅ Exporté: monthly_category_trends_20260215.csv
✅ Exporté: activity_analysis_20260215.csv
✅ Exporté: activity_by_status_20260215.csv
✅ Exporté: activity_by_type_20260215.csv
✅ Exporté: monthly_activities_20260215.csv

📂 Tous les exports dans: /Users/jo/Desktop/BNP Paribas/Data/Exports


In [92]:
# Fermer la connexion
conn.close()
print("✅ Connexion fermée")

✅ Connexion fermée


---
## 13. SR avec 0 Interactions (Contacts)

**Analyse** : Identifier les SR qui n'ont eu aucune interaction client (CONTACT_)

In [93]:
# Reconnecter à la base de données si nécessaire
conn = sqlite3.connect(str(DB_PATH))

# SR avec 0 interactions par catégorie
zero_contact_analysis = pd.read_sql_query("""
SELECT 
    c.NAME as category_name,
    COUNT(sr.ID) as total_srs,
    COUNT(CASE WHEN (SELECT COUNT(*) FROM srcontact WHERE srcontact.SR_ID = sr.ID) = 0 THEN 1 END) as srs_zero_contact,
    ROUND(COUNT(CASE WHEN (SELECT COUNT(*) FROM srcontact WHERE srcontact.SR_ID = sr.ID) = 0 THEN 1 END) * 100.0 / COUNT(sr.ID), 2) as pct_zero_contact,
    ROUND(AVG(CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL)), 2) as avg_hours_all,
    ROUND(AVG(CASE WHEN (SELECT COUNT(*) FROM srcontact WHERE srcontact.SR_ID = sr.ID) = 0 
                   THEN CAST((julianday(sr.CLOSINGDATE) - julianday(sr.CREATIONDATE)) * 24 AS REAL) END), 2) as avg_hours_zero_contact
FROM sr
LEFT JOIN category c ON sr.CATEGORY_ID = c.ID
WHERE strftime('%Y-%m', sr.CREATIONDATE) BETWEEN '2025-01' AND '2025-09'
  AND sr.CLOSINGDATE IS NOT NULL
GROUP BY c.ID, c.NAME
HAVING COUNT(sr.ID) > 100
ORDER BY srs_zero_contact DESC
LIMIT 30
""", conn)

print("="*70)
print("📊 SR AVEC 0 INTERACTIONS (CONTACT) PAR CATÉGORIE")
print("="*70)
print("\nCatégories avec le plus de SR sans interaction client\n")

display(zero_contact_analysis)

# Statistiques globales
total_zero_contact = zero_contact_analysis['srs_zero_contact'].sum()
total_srs_analyzed = zero_contact_analysis['total_srs'].sum()
global_pct = (total_zero_contact / total_srs_analyzed) * 100

print(f"\n📊 Statistiques (catégories >100 SRs) :")
print(f"   • Total SRs avec 0 contact : {total_zero_contact:,} / {total_srs_analyzed:,}")
print(f"   • Taux global : {global_pct:.2f}%")
print(f"   • Temps moyen (SRs avec 0 contact) : {zero_contact_analysis['avg_hours_zero_contact'].mean():.1f}h ({zero_contact_analysis['avg_hours_zero_contact'].mean()/24:.1f}j)")

📊 SR AVEC 0 INTERACTIONS (CONTACT) PAR CATÉGORIE

Catégories avec le plus de SR sans interaction client



,category_name,total_srs,srs_zero_contact,pct_zero_contact,avg_hours_all,avg_hours_zero_contact
0,Check Lists,18431,14130,76.66,4.32,1.22
1,Check list,5909,3935,66.59,6.38,3.66
2,Income/Redemptions,14637,3191,21.80,168.75,84.35
3,Tax,42073,2285,5.43,325.95,80.92
4,Checklist,3046,1970,64.67,26.11,21.02
5,Settlement Trade Processing,11726,1807,15.41,43.22,5.29
6,CA - Payment,9991,1731,17.33,169.08,74.99
7,BAU,20128,1655,8.22,65.45,157.03
8,Cash instruction,65135,1625,2.49,51.76,8.87
9,CREST,31657,1550,4.90,28.30,9.11



📊 Statistiques (catégories >100 SRs) :
   • Total SRs avec 0 contact : 44,347 / 460,069
   • Taux global : 9.64%
   • Temps moyen (SRs avec 0 contact) : 53.6h (2.2j)


In [94]:
# Visualisation : SR avec 0 contact
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Top 15: SR avec 0 Contact (Volume)', 'Taux de SR sans Contact (%)'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

df_viz = zero_contact_analysis.head(15).copy()

# Graphique 1 : Volume de SRs avec 0 contact
fig.add_trace(
    go.Bar(
        y=df_viz['category_name'],
        x=df_viz['srs_zero_contact'],
        orientation='h',
        text=df_viz['srs_zero_contact'].apply(lambda x: f"{x:,}"),
        textposition='outside',
        marker_color='lightcoral',
        name='SRs sans contact',
        hovertemplate='<b>%{y}</b><br>SRs sans contact: %{x:,}<extra></extra>'
    ),
    row=1, col=1
)

# Graphique 2 : Pourcentage de SRs avec 0 contact
df_sorted = df_viz.sort_values('pct_zero_contact')
fig.add_trace(
    go.Bar(
        y=df_sorted['category_name'],
        x=df_sorted['pct_zero_contact'],
        orientation='h',
        text=df_sorted['pct_zero_contact'].apply(lambda x: f"{x:.1f}%"),
        textposition='outside',
        marker_color=df_sorted['pct_zero_contact'],
        marker_colorscale='RdYlGn_r',
        name='% sans contact',
        hovertemplate='<b>%{y}</b><br>Taux: %{x:.1f}%<extra></extra>'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Nombre de SRs", row=1, col=1)
fig.update_yaxes(title_text="Catégorie", row=1, col=1)
fig.update_xaxes(title_text="% de SRs sans contact", row=1, col=2)
fig.update_yaxes(title_text="Catégorie", row=1, col=2)

fig.update_layout(height=600, showlegend=False, title_text="Analyse des SR avec 0 Interaction Client")
fig.show()

In [95]:
# Fermer la connexion
conn.close()
print("✅ Connexion fermée")

✅ Connexion fermée
